In [3]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
import pandas as pd

In [66]:
df = pd.read_csv("spam.csv",encoding="latin")
input_texts = df['v2']
labels = df['v1']
df.iloc[2]['v2']

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [5]:
!head edger_allan.txt

'head' is not recognized as an internal or external command,
operable program or batch file.


input_texts = []
labels = []
input_files = [
  'edger-allen.txt',
  'robert-frost.txt',
]
for label,f in enumerate(input_files): #enumerate on list give you index 0 1 so force and also actual val
    for line in open(f):
        line = line.rstrip().lower() #trimming
        if line: # to make sure line is not empty
            line = line.translate(str.maketrans('', '', string.punctuation)) #remove punctiations

            input_texts.append(line)
            labels.append(label)
            

In [6]:
input_texts

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [7]:
labels

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [9]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [10]:
len(Ytrain) 

4179

In [11]:
len(Ytest)

1393

In [12]:
idx =1 
word2idx = {"<unk>":0}
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [13]:
word2idx

{'<unk>': 0,
 'URGENT!!': 1,
 'Your': 2,
 '4*': 3,
 'Costa': 4,
 'Del': 5,
 'Sol': 6,
 'Holiday': 7,
 'or': 8,
 'å£5000': 9,
 'await': 10,
 'collection.': 11,
 'Call': 12,
 '09050090044': 13,
 'Now': 14,
 'toClaim.': 15,
 'SAE,': 16,
 'TC': 17,
 's,': 18,
 'POBox334,': 19,
 'Stockport,': 20,
 'SK38xh,': 21,
 'Costå£1.50/pm,': 22,
 'Max10mins': 23,
 'G': 24,
 'says': 25,
 'you': 26,
 'never': 27,
 'answer': 28,
 'your': 29,
 'texts,': 30,
 'confirm/deny': 31,
 'Sitting': 32,
 'in': 33,
 'mu': 34,
 'waiting': 35,
 'for': 36,
 'everyone': 37,
 'to': 38,
 'get': 39,
 'out': 40,
 'of': 41,
 'my': 42,
 'suite': 43,
 'so': 44,
 'I': 45,
 'can': 46,
 'take': 47,
 'a': 48,
 'shower': 49,
 'sry': 50,
 "can't": 51,
 'talk': 52,
 'on': 53,
 'phone,': 54,
 'with': 55,
 'parents': 56,
 "I'm": 57,
 'eatin': 58,
 'now': 59,
 'lor,': 60,
 'but': 61,
 'goin': 62,
 'back': 63,
 'work': 64,
 'soon...': 65,
 'E': 66,
 'mountain': 67,
 'deer': 68,
 'show': 69,
 'huh...': 70,
 'watch': 71,
 'b4': 72,
 'liao,

In [14]:
len(word2idx)

13142

In [15]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
    #here the part we handel unknown words and note we did not add to train as word2idx bulit on training data and all words in training we know it
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)
train_text_int[100:105]

[[388, 927, 472, 107, 48, 549, 928, 47, 929],
 [511, 243, 675, 930, 931, 38, 932],
 [933,
  934,
  935,
  83,
  936,
  8,
  937,
  33,
  579,
  33,
  172,
  938,
  939,
  940,
  297,
  941,
  942,
  166,
  297,
  943,
  38,
  944,
  225,
  791,
  460,
  26,
  228,
  945],
 [57,
  603,
  118,
  489,
  946,
  225,
  135,
  409,
  82,
  38,
  52,
  350,
  349,
  947,
  948,
  949,
  950,
  470,
  951,
  952,
  953],
 [954,
  107,
  955,
  88,
  751,
  956,
  171,
  172,
  957,
  958,
  493,
  959,
  960,
  961,
  297,
  962,
  963,
  964,
  568,
  221,
  39,
  965,
  221,
  966,
  967,
  968,
  55,
  969,
  89,
  800,
  766,
  970]]

In [16]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)


In [17]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0) #here we populate a0 and by0 with class 0
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1) #here we populate a1 and by1 with class 1

In [18]:
A0[111]

array([1., 1., 1., ..., 1., 1., 1.])

In [19]:
A0

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [20]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [21]:
print(A0[0])

[7.6091919e-05 7.6091919e-05 7.6091919e-05 ... 7.6091919e-05 7.6091919e-05
 7.6091919e-05]


In [22]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)


In [23]:
logA0

array([[-9.48356849, -9.48356849, -9.48356849, ..., -9.48356849,
        -9.48356849, -9.48356849],
       [-9.48356849, -9.48356849, -9.48356849, ..., -9.48356849,
        -9.48356849, -9.48356849],
       [-9.48531707, -9.48531707, -9.48531707, ..., -9.48531707,
        -9.48531707, -9.48531707],
       ...,
       [-9.48356849, -9.48356849, -9.48356849, ..., -9.48356849,
        -9.48356849, -9.48356849],
       [-9.48364458, -9.48364458, -9.48364458, ..., -9.48364458,
        -9.48364458, -9.48364458],
       [-9.48356849, -9.48356849, -9.48356849, ..., -9.48356849,
        -9.48356849, -9.48356849]])

In [24]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.867910983488873, 0.13208901651112706)

In [25]:
# build a classifier
#the class go on each list in text_int and count probabilty in A0,A1,PI0,PI1 and add prior and see which class on top
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):#take as input a line of text repesented as text and class that indicate which markov model to use
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [26]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [27]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.998085666427375


In [28]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.9691313711414213


In [29]:
from sklearn.metrics import confusion_matrix, f1_score

In [30]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[3627,    0],
       [   8,  544]], dtype=int64)

In [31]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[1197,    1],
       [  42,  153]], dtype=int64)

In [32]:
f1_score(Ytrain, Ptrain)

0.9927007299270074

In [33]:
f1_score(Ytest, Ptest)

0.8767908309455588

In [88]:
test_text2 = "Congratulations on your graduation! We wish you success in your future endeavors. Stay connected with our alumni network."




      
tokens = test_text2.split()
print(tokens)
for tok in tokens:
    print(tok)
    
     #here the part we handel unknown words and note we did not add to train as word2idx bulit on training data and all words in training we know it
line_as_int2 = [word2idx.get(token, 0) for token in tokens]
print(line_as_int2)


        
    
    
   
        




['Congratulations', 'on', 'your', 'graduation!', 'We', 'wish', 'you', 'success', 'in', 'your', 'future', 'endeavors.', 'Stay', 'connected', 'with', 'our', 'alumni', 'network.']
Congratulations
on
your
graduation!
We
wish
you
success
in
your
future
endeavors.
Stay
connected
with
our
alumni
network.
[1475, 53, 29, 0, 346, 515, 26, 6516, 33, 29, 454, 0, 0, 0, 55, 708, 0, 9842]


In [89]:
line_as_int2

[1475, 53, 29, 0, 346, 515, 26, 6516, 33, 29, 454, 0, 0, 0, 55, 708, 0, 9842]

In [90]:
Ptest = clf.predict([line_as_int2])
Ptest

array([0.])